# baseline v3

이 베이스라인 코드는 `사전학습 모델 로드`, `배치 학습`, `파인튜닝`, `양자화`, `PEFT` 등이 적용된 버전입니다.

Colab의 GPU 환경에서 개발되었습니다.
- 런타임 - 런타임 유형 변경 - GPU로 변경(T4 GPU 등)



# 환경 준비

개발 환경에 필요한 라이브러리 버전을 고정하고 최신 버전으로 라이브러리를 업데이트합니다.

- 아래 셀 실행
- 실행 완료 후 런타임 - 세션 다시 시작

In [28]:
# !pip -q install "transformers>=4.44.2" "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.43.1" datasets pillow pandas torch torchvision --upgrade

# 데이터 준비

개발에 필요한 데이터를 준비합니다.

- train.csv, train 폴더
- test.csv, test 폴더
- sample_submission.csv

본 베이스라인은 colab에서 구글 드라이브를 마운트하여 사용합니다.

데이터를 압축 해제하는데 몇 분 정도의 시간이 소요됩니다.

#### 실습 참고 내용

    챕터 2-2 합성 데이터 실습
    - 구글 드라이브 마운트 : drive()

In [29]:
# # 구글드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

In [30]:
# # 압축 해제
# !unzip "/content/drive/My Drive/250918/data.zip" -d "/content/"

# 라이브러리, 데이터, 설정

In [ ]:
import os, re, math, random
import pandas as pd
cache_root = "/home/team138/.cache/huggingface"
os.environ["TMPDIR"] = cache_root
os.environ["HF_HOME"] = cache_root
os.environ["HUGGINGFACE_HUB_CACHE"] = cache_root
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_root, "modules")
os.environ["TRANSFORMERS_CACHE"] = cache_root
os.environ["HF_DATASETS_CACHE"] = cache_root
os.makedirs(cache_root, exist_ok=True)
os.makedirs(os.environ["HF_MODULES_CACHE"], exist_ok=True)
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import torch
from typing import Dict, List, Any
from transformers import (
    AutoModelForVision2Seq,
    AutoProcessor,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 사전 학습 모델 정의
MODEL_ID = "OpenGVLab/InternVL2-8B"
IMAGE_SIZE = 384
MAX_NEW_TOKENS = 8
SEED = 42
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# 데이터셋 로드
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")

# 학습데이터 200개만 추출
train_df = train_df.sample(n=200, random_state=SEED).reset_index(drop=True)

/home/team138/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/team138/venv/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Device: cuda


In [ ]:
from transformers import AutoTokenizer, AutoProcessor, AutoModelForCausalLM

model_id = "OpenGVLab/InternVL2-8B"
hf_token = ""  # 공개 모델이면 None 또는 인자 제거

tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token,
                                          cache_dir=cache_root, trust_remote_code=True)
processor = AutoProcessor.from_pretrained(model_id, token=hf_token,
                                          cache_dir=cache_root, trust_remote_code=True)

print(type(processor))
print(getattr(processor, "image_token", None))

<class 'transformers_modules.OpenGVLab.InternVL2_hyphen_8B.6fb9ad6924f69424e57fab2ab061d707688f0296.tokenization_internlm2.InternLM2Tokenizer'>
None


# 모델, Processor

7.5GB 정도의 모델 다운로드가 진행됩니다. 10~20분 정도가 소요됩니다.

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - LoRA 구현 : LoraConfig()

In [6]:
# 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 프로세서
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    min_pixels=IMAGE_SIZE*IMAGE_SIZE,
    max_pixels=IMAGE_SIZE*IMAGE_SIZE,
    trust_remote_code=True,
)

# 사전학습 모델
from transformers import AutoModelForCausalLM
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# 양자화 모델로 로드
base_model = prepare_model_for_kbit_training(base_model)
base_model.gradient_checkpointing_enable()

cfg = base_model.config
task_type = "CAUSAL_LM"

# LoRA 적용 후보 모듈 확인
print("=== LoRA 적용 가능 모듈 후보 ===")
for n, m in base_model.named_modules():
    if any(k in n for k in ["q_proj","k_proj","v_proj","o_proj","proj","gate","up","down","fc","linear"]):
        print(n, type(m))

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["attn.proj", "mlp.fc1", "mlp.fc2"],  # 실제 모델 모듈 이름
    task_type="CAUSAL_LM",
)

# PEFT 모델 생성
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

InternLM2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards: 100%|██████████| 4/4 [00:20<00:00,  5.01s/it]


=== LoRA 적용 가능 모듈 후보 ===
vision_model.encoder.layers.0.attn.proj_drop <class 'torch.nn.modules.dropout.Dropout'>
vision_model.encoder.layers.0.attn.proj <class 'bitsandbytes.nn.modules.Linear4bit'>
vision_model.encoder.layers.0.mlp.fc1 <class 'bitsandbytes.nn.modules.Linear4bit'>
vision_model.encoder.layers.0.mlp.fc2 <class 'bitsandbytes.nn.modules.Linear4bit'>
vision_model.encoder.layers.1.attn.proj_drop <class 'torch.nn.modules.dropout.Dropout'>
vision_model.encoder.layers.1.attn.proj <class 'bitsandbytes.nn.modules.Linear4bit'>
vision_model.encoder.layers.1.mlp.fc1 <class 'bitsandbytes.nn.modules.Linear4bit'>
vision_model.encoder.layers.1.mlp.fc2 <class 'bitsandbytes.nn.modules.Linear4bit'>
vision_model.encoder.layers.2.attn.proj_drop <class 'torch.nn.modules.dropout.Dropout'>
vision_model.encoder.layers.2.attn.proj <class 'bitsandbytes.nn.modules.Linear4bit'>
vision_model.encoder.layers.2.mlp.fc1 <class 'bitsandbytes.nn.modules.Linear4bit'>
vision_model.encoder.layers.2.mlp.fc2 <cl

# 프롬프트 템플릿

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - 프롬프트 템플릿 : convert_to_chatml(), formatting_prompts_func()

In [7]:
# 모델 지시사항
SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)

# 프롬프트
def build_mc_prompt(question, a, b, c, d):
    return (
        f"{question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
    )

# Custom Dataset, Collator

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - TensorDataset()

    챕터 5-2 데이터 생성 및 파인튜닝 (향후 학습 분량)
    - IntentDataset()

In [22]:
from dataclasses import dataclass
from typing import Any
from torch.utils.data import Dataset
from PIL import Image

# 커스텀 데이터셋
class VQAMCDataset(Dataset):
    def __init__(self, df, tokenizer, train: bool = True):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.train = train

        specials = tokenizer.additional_special_tokens or []
        self.image_placeholder = "<image>"
        for tok in specials:
            if tok.lower().startswith("<img"):
                self.image_placeholder = tok
                break

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        image = Image.open(row["path"]).convert("RGB")
        prompt = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])

        messages = [
            {"role": "system", "content": SYSTEM_INSTRUCT},
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": prompt},
                ],
            },
        ]
        if self.train:
            gold = str(row["answer"]).strip().lower()
            messages.append({"role": "assistant", "content": gold})

        return {"messages": messages, "image": image}


# 데이터 콜레이터
@dataclass
class DataCollator:
    processor: Any
    train: bool = True
    add_generation_prompt: bool = False

    def __call__(self, batch):
        texts, images = [], []
        for sample in batch:
            flat_messages = []
            image_for_sample = None

            for message in sample["messages"]:
                content = message["content"]
                if isinstance(content, list):
                    pieces = []
                    for item in content:
                        if item.get("type") == "text":
                            pieces.append(item["text"])
                        elif item.get("type") == "image":
                            image_for_sample = item["image"]
                            pieces.append("<image>")
                    flat_messages.append(
                        {"role": message["role"], "content": "\n".join(pieces)}
                    )
                else:
                    flat_messages.append(message)

            if image_for_sample is None:
                raise ValueError("이미지가 포함된 turn이 없습니다.")

            text = self.processor.apply_chat_template(
                flat_messages,
                tokenize=False,
                add_generation_prompt=self.add_generation_prompt,
            )
            texts.append(text)
            images.append(image_for_sample)

        encoded = self.processor(
            text=texts,
            images=images,
            padding=True,
            return_tensors="pt",
        )
        encoded.pop("inputs_embeds", None)
        if self.train:
            encoded["labels"] = encoded["input_ids"].clone()
        return encoded


# DataLoader

#### 실습 참고 내용

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 데이터로더 정의 : DataLoader()

In [23]:
from sklearn.model_selection import train_test_split

# 검증용 데이터 분리
split = int(len(train_df)*0.9)
train_subset, valid_subset = train_test_split(train_df, test_size=0.1, random_state=SEED, stratify=train_df["answer"])

# VQAMCDataset 형태로 변환
train_ds = VQAMCDataset(train_subset, tokenizer, train=True)
valid_ds = VQAMCDataset(valid_subset, tokenizer, train=False)

# 데이터로더
train_loader = DataLoader(
    train_ds,
    batch_size=1,
    shuffle=True,
    collate_fn=DataCollator(processor, train=True, add_generation_prompt=False),
    num_workers=0,
)
valid_loader = DataLoader(
    valid_ds,
    batch_size=1,
    shuffle=False,
    collate_fn=DataCollator(processor, train=False, add_generation_prompt=False),
    num_workers=0,
)


# fine-tuning

- 200개만 학습 : 10~20분 소요

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - 모델 정의 : SimpleMLP(), SequentialMLP()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [24]:
from tqdm.auto import tqdm

EPOCHS = 1
GRAD_ACCUM = 4
grad_scaler_enabled = torch.cuda.is_available()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
updates_per_epoch = math.ceil(len(train_loader) / GRAD_ACCUM)
total_training_steps = EPOCHS * updates_per_epoch
warmup_steps = int(total_training_steps * 0.03)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps,
)

scaler = torch.cuda.amp.GradScaler(enabled=grad_scaler_enabled)

global_step = 0
for epoch in range(EPOCHS):
    running = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1} [train]", unit="batch")
    last_step = 0

    for step, batch in enumerate(progress_bar, start=1):
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        batch.pop("inputs_embeds", None)

        if "pixel_values" not in batch:
            raise ValueError("pixel_values missing from batch; check DataCollator output.")

        forward_inputs = {
            k: batch[k]
            for k in ("pixel_values", "input_ids", "attention_mask", "position_ids", "image_flags", "labels")
            if k in batch
        }
        forward_inputs["return_dict"] = True

        with torch.cuda.amp.autocast(enabled=grad_scaler_enabled, dtype=torch.float16):
            outputs = model.base_model.model(**forward_inputs)
            loss = outputs.loss / GRAD_ACCUM

        scaler.scale(loss).backward()
        running += loss.item()
        last_step = step

        if step % GRAD_ACCUM == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1

            avg_loss = running / GRAD_ACCUM
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})
            running = 0.0

    if last_step % GRAD_ACCUM != 0:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()
        global_step += 1

    model.eval()
    val_loss = 0.0
    val_steps = 0
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=grad_scaler_enabled, dtype=torch.bfloat16):
        for vb in tqdm(valid_loader, desc=f"Epoch {epoch + 1} [valid]", unit="batch"):
            vb = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in vb.items()}
            vb.pop("inputs_embeds", None)

            forward_inputs = {
                k: vb[k]
                for k in ("pixel_values", "input_ids", "attention_mask", "position_ids", "image_flags", "labels")
                if k in vb
            }
            forward_inputs["return_dict"] = True

            outputs = model.base_model.model(**forward_inputs)
            val_loss += outputs.loss.item()
            val_steps += 1

    print(f"[Epoch {epoch + 1}/{EPOCHS}] valid loss {val_loss / val_steps:.4f}")
    model.train()

SAVE_DIR = "./models/OpenGVLabInternVL2_8B"
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)
print("Saved:", SAVE_DIR)

/tmp/ipykernel_8342/215713023.py:18: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=grad_scaler_enabled)
Epoch 1 [train]:   0%|          | 0/180 [00:00<?, ?batch/s]


ValueError: pixel_values missing from batch; check DataCollator output.

# inference

30분~1시간 소요

#### 실습 참고 내용

    챕터4-1 RAG 기반 Customer Service AI 에이전트 개발
    - 데이터 파서 : langchain_core.output_parsers(), StrOutputParser()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [ ]:
# 데이터 파서 : 모델의 응답에서 선지를 추출
def extract_choice(text: str) -> str:
    text = text.strip().lower()

    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines:
        return "a"
    last = lines[-1]
    if last in ["a", "b", "c", "d"]:
        return last

    tokens = last.split()
    for tok in tokens:
        if tok in ["a", "b", "c", "d"]:
            return tok
    return "a"

# 테스트 데이터 준비
test_ds = VQAMCDataset(test_df, processor, train=False)
test_loader = DataLoader(
    test_ds,
    batch_size=4,  # GPU 여유에 맞춰 조절
    shuffle=False,
    collate_fn=DataCollator(processor, train=False, add_generation_prompt=True),
    num_workers=0,
)

model.eval()
preds = []

for batch in tqdm(test_loader, desc="Inference", unit="batch"):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        out_ids = model.generate(
            **batch,
            max_new_tokens=2,
            do_sample=False,
            eos_token_id=processor.tokenizer.eos_token_id,
        )
    decoded = processor.batch_decode(out_ids, skip_special_tokens=True)
    preds.extend(extract_choice(text) for text in decoded)

submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
submission.to_csv("/content/submission.csv", index=False)
print("Saved /content/submission.csv")

Inference:   0%|          | 0/972 [00:00<?, ?batch/s]c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-06` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Inference:   4%|▎         | 35/972 [16:38<7:25:36, 28.53s/batch]


KeyboardInterrupt: 

In [ ]:
# 모델 응답 예시
# print(output_text)

system
You are a helpful visual question answering assistant. Answer using exactly one letter among a, b, c, or d. No explanation.
user
이 사진의 주요 상황은 무엇인가요?
(a) 수업 시간에 공부하고 있다
(b) 회의에 참석하고 있다
(c) 졸업식 준비 중이다
(d) 시험을 치르고 있다

정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요.
assistant
c


In [ ]:
import torch, sys
print("PY:", sys.executable)
print("torch:", torch.__version__)
print("torch.version.cuda:", torch.version.cuda)
print("is_available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

PY: c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\python.exe
torch: 2.6.0+cu124
torch.version.cuda: 12.4
is_available: True
GPU: NVIDIA GeForce RTX 3050 OEM


"c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\python.exe" -m pip uninstall -y torch torchvision torchaudio
"c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\python.exe" -m pip install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio